# Page view approach - economic recovery WUJ

This approach does not rely on the existing knowledge graph. A functional graph based on page hit session data is created, before further filtering of the graph, to find a list of pages related to the economic recovery whole user journey (WUJ).   

In [ ]:
from neo4j import GraphDatabase
import networkx as nx
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import operator
from operator import itemgetter
from collections import defaultdict
import gspread 
from oauth2client.service_account import ServiceAccountCredentials

## Functions for coercing knowledge graph into NetworkX

In [ ]:
def getSubgraph(q, parameters=None):

    '''
    Given a Cypher query q, this function queries the knowledge graph,
    returns the nodes and edges from this query, and uses them to construct
    a networkx graph.

    E.g. getSubgraph(r'MATCH (u:Cid)-[r:HYPERLINKS_TO]->(v:Cid) RETURN *')
         returns the structural graph.

    Optionally, can add in parameters (dictionary), allowing Python variables
    to be integrated into the Cypher query q.

    E.g.
        parameters = {}
        parameters['pages'] = ['a','list','of','stuff']
        q7 = f"""
        MATCH (u:Cid)-[r]-(v:Cid)
        WHERE u.name IN $pages AND v.name in $pages
        RETURN *
        """

        g7 = getSubgraph(q7, parameters)
    '''

    # get credentials
    # add to .secrets: export KG_PWD="<PASSWORD>"
    KG_PWD = os.getenv("KG_PWD")

    # create connection to knowledge graph
    driver = GraphDatabase.driver(
        "bolt+s://knowledge-graph.integration.govuk.digital:7687",
        auth=("neo4j", KG_PWD),
    )

    # run query on knowledge graph
    results = driver.session().run(q, parameters)

    # create networkx graph object
    G = nx.MultiDiGraph()

    # add nodes into networkx graph object
    nodes = list(results.graph()._nodes.values())
    print("Adding nodes\n")
    for node in tqdm(nodes):
        G.add_node(node.id, labels=node._labels, properties=node._properties)

    # add edges into networkx graph object
    rels = list(results.graph()._relationships.values())
    print("Adding edges\n")
    for rel in tqdm(rels):
        G.add_edge(
            rel.start_node.id,
            rel.end_node.id,
            key=rel.id,
            type=rel.type,
            properties=rel._properties,
        )

    return G


def showGraph(g):
    """
    Given a networkx graph g, this function visualises the graph.
    Do not use for a large g.
    """
    print(nx.info(g))
    nx.draw(g)

In [ ]:
def getNoOfTruePages(g):
    """
    Calculate a proxy recall metric for the list of pages identified in a
    subgraph (when compared to the ground truth for the economic recovery pages). 
    The output is the number of pages in the subgraph list that are present in 
    the ground truth list.  
    """
    
    # convert nodeIds to page path slug for the subgraph list
    subgraph_list = [node[0] for node in g.nodes(data=True)]

    # set up the ground truth list
    true_list = list(economic_pages)

    # how many pages are in the subgraph list that are also in the ground truth list
    return (len(true_list)) - (len([node for node in true_list if node not in subgraph_list]))

## Pre-defined economic recovery subgraph

Following the page view approach, import the pages (nodes) in the economic recovery subgraph

In [ ]:
GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

# Connect to service account
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_APPLICATION_CREDENTIALS, scope) 
gc = gspread.authorize(credentials)

# Import the data from google sheets
spreadsheet_key = '1lLsgQRsl4bXmbyiwrbMNQOR_Zu-FkbqYRUmOOhTYc70' 
book = gc.open_by_key(spreadsheet_key) 
worksheet = book.worksheet('Top pages') 
table = worksheet.get_all_values()

# Convert table data into a dataframe then set 
df = pd.DataFrame(table[1:], columns=table[0])
economic_pages=set(list(filter(None, df['pagePathv2'])))